# TO-DO
1. (DONE) Make config updates recursive. Currently nested dicts are just overwriting. That is, if two config files have a MODEL keyword, the whole model is being overwritten.
2. switch model to `**kwargs`-based setup so we can list all possibly-relevant ones.
3. more-efficient CSV processing

In [1]:
import os
os.chdir('/home/djliden91/git/projects/numerai')
from src.config import config
import yaml
from pathlib import Path
from src.models import fastai_tabular
import sys
import pandas as pd
import numpy as np
import time
from src.config.config import Config
#from fastai.tabular.all import *
from importlib import import_module
from pathlib import Path

#from src.utils.eval import FastSubmission
from src.utils.setup import credential
from src.utils.setup import download_current
from src.utils.setup import process_current
from src.utils.setup import init_numerapi
from src.utils.cross_val import EraCV

from src.utils.eval import save_preds

#import src.models

from src.utils.metrics import sharpe, val_corr

# SET THIS TORCH STUFF IN FASTAI SECTION
# set flags / seeds
import gc
#torch.backends.cudnn.benchmark = True
np.random.seed(1)
#torch.manual_seed(1)
#torch.cuda.manual_seed(1)

In [2]:
model = "fastai_tabular"
configpath = "./src/config/experiments/config_debug.yaml"
default_config = Path("./src/config/default_config.yaml")
cfg = Config(default_config)
model_cfg = Path(f'./src/models/default_configs/{model}.yaml')
model_cls = model.title().replace("_","")
cfg.update_config(model_cfg)
cfg.update_config(configpath)
ct = time.localtime()
current_time = f'{ct[0]}_{ct[1]}_{ct[2]}_{ct[3]}{ct[4]}'
cfg.update_config({'SYSTEM':{'TIME':current_time}})
print(cfg)


CV:
  GAP: 0
  TRAIN_START: 0
  TRAIN_STOP: null
  VAL_END: 210
  VAL_N_ERAS: 3
  VAL_START: 208
DATA:
  REFRESH: false
  SAVE_PROCESSED_TRAIN: true
EVAL:
  CHUNK_SIZE: 1000000
  SAVE_PREDS: false
  SUBMIT_PREDS: false
MODEL:
  BATCHNORM_FINAL: true
  BATCH_SIZE: 2048
  DROPOUT_P: 0
  LAYERS:
  - 400
  - 400
  LEARNING_RATE: null
  LOSS_FUNCTION: MSELossFlat
  METRICS:
  - SpearmanCorrCoef
  NAME: Unnamed
  N_EPOCHS: 1
  TYPE: fastai_tabular
  USE_BATCHNORM: true
  WEIGHT_DECAY: 0
  Y_RANGE:
  - 0
  - 1
SYSTEM:
  DEBUG: false
  TIME: '2021_4_3_753'



In [3]:
credential()
napi = init_numerapi()

Loaded Numerai Public Key into Global Environment!
Loaded Numerai Secret Key into Global Environment!


In [4]:
# 2. Define key paths
round = napi.get_current_round()
#current_file = Path(data_dir/f"numerai_dataset_{round}.zip")
train = Path(f"./input/numerai_dataset_{round}/numerai_training_data.csv")
tourn = Path(f"./input/numerai_dataset_{round}/numerai_tournament_data.csv")
processed = Path('./input/training_processed.csv')
processed_pkl = Path('./input/training_processed.pkl')
output = Path("./output/")

In [5]:
%%time
download_current(napi=napi)
training_data, feature_cols, target_cols = process_current(processed,
                                                           processed_pkl, train, tourn)

The dataset has already been downloaded.
You can re-download it with refresh = True
Loading the pickled training data from file

CPU times: user 114 ms, sys: 1.21 s, total: 1.32 s
Wall time: 2.88 s


time with float16: 1 min 28 s
Time with float32: 1 min 12 s
time with default: 1 min 32 s
time with dask: 56.2 s
time with dask and read train csv with dask: 1.16 s
time deferring compute to after whole concat done: 52.9 s

In [6]:
print(cfg.MODEL)
model_cls

BATCHNORM_FINAL: true
BATCH_SIZE: 2048
DROPOUT_P: 0
LAYERS:
- 400
- 400
LEARNING_RATE: null
LOSS_FUNCTION: MSELossFlat
METRICS:
- SpearmanCorrCoef
NAME: Unnamed
N_EPOCHS: 1
TYPE: fastai_tabular
USE_BATCHNORM: true
WEIGHT_DECAY: 0
Y_RANGE:
- 0
- 1



'FastaiTabular'

In [7]:
# Load Model
modmod = import_module(f'src.models.{model}')
mod = getattr(modmod, model_cls)(**cfg.MODEL.config)

In [8]:
era_split = EraCV(eras = training_data.era)

In [9]:
era_split

EraCV(last era:212)

In [10]:
train_idx, test_idx = era_split.get_splits(valid_start = 209,
                                           valid_n_eras = cfg.CV.VAL_N_ERAS,
                                           train_start = cfg.CV.TRAIN_START,
                                           train_stop = cfg.CV.TRAIN_STOP)

In [11]:
mod.dls = mod.build_data_loaders(df = training_data, cont_names = list(feature_cols),
          train_idx = list(train_idx), val_idx = list(test_idx))
mod.init_learner()

In [12]:
print(mod.learner.lr)

None


In [13]:
mod.fit(df = training_data, cont_names = list(feature_cols),
          train_idx = list(train_idx), val_idx = list(test_idx))

training the model

[N | train --------- | valid ----------- | corr ------------- | time]


AssertionError: Trying to set 0 values for lr but there are 1 parameter groups.

In [12]:
val_preds = mod.predict(training_data.iloc[test_idx])

prediction = val_preds
target = list(training_data.target.iloc[test_idx])
val_era = list(training_data.era[test_idx])
eval_df = pd.DataFrame({'prediction':prediction,
                        'target':target, 'era':val_era})
eval_df
print(sharpe(eval_df))
print(val_corr(eval_df))

0.8501802522929379
0.014772592710707605


In [13]:
%%script false --no-raise-error
# might be necessary
print("training the model\n")
print(("[N | train --------- | valid ----------- |"
       " corr ------------- | time]"))
start = time.time()
with learn.no_bar():
    learn.fit_one_cycle(n_epoch = cfg.TRAIN.N_EPOCHS,
                        wd = cfg.MODEL.WEIGHT_DECAY)
end = time. time()


In [14]:
save_preds(model=mod, chunksize=1e6,
           pred_path = output/"preds.csv",
           feature_cols = feature_cols, tourn_path=tourn)

In [26]:
%%time
# dask
import dask.dataframe as dd
df = dd.read_csv(tourn)
val_df = df[df['data_type'] == 'validation']


CPU times: user 220 ms, sys: 11.2 ms, total: 232 ms
Wall time: 114 ms


In [28]:
%%time
val_df = val_df.compute()

CPU times: user 1min 27s, sys: 9.54 s, total: 1min 37s
Wall time: 34.8 s


,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.00,0.75,0.50,...,0.75,0.75,1.00,0.75,0.50,0.50,1.00,0.00,0.00,0.25
1,n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.50,0.00,0.00,...,0.50,0.50,0.75,1.00,0.75,0.50,0.50,0.50,0.50,0.50
2,n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.00,1.00,1.00,...,0.00,0.00,0.50,0.25,0.00,0.00,0.50,0.50,0.00,1.00
3,n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.00,0.00,0.00,...,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.50
4,n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.00,0.50,0.50,...,0.50,0.75,0.00,0.00,0.75,1.00,0.00,0.25,1.00,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17019,nffb61f786fe2a96,era212,validation,0.00,0.50,0.75,0.00,0.50,0.50,0.50,...,1.00,1.00,1.00,1.00,1.00,0.75,0.75,0.75,1.00,0.50
17020,nffbe101615ad597,era212,validation,0.75,0.75,0.75,0.75,0.25,0.25,0.25,...,0.75,0.75,0.25,0.25,0.75,0.75,0.25,0.50,0.75,0.25
17021,nffc1dc801a3318a,era212,validation,0.25,1.00,0.50,0.00,0.00,0.25,0.50,...,0.25,0.50,0.00,0.00,0.25,0.25,0.00,0.75,0.75,0.50
17022,nffc376c3127112d,era212,validation,0.00,0.75,0.75,0.00,0.75,0.50,0.75,...,0.50,0.50,0.75,0.75,0.25,0.25,0.75,0.50,0.50,1.00


In [25]:
del df
del pdf

NameError: name 'pdf' is not defined